In [ ]:
!pip install objectnat
# Installing the 'objectnat' library which provides functions for working with multimodal graphs and adjacency matrices

In [ ]:
!pip install pyarrow 
# Installing 'pyarrow' to read and write .parquet files, a columnar storage file format.

In [8]:
data_path = 'F:/Резерв/Учеба/НИР-3/Чат gpt'

In [9]:
import pickle
import networkx as nx

In [11]:
G_intermodal = pickle.load(open('F:/Резерв/Учеба/НИР-3/Чат gpt/graph.pickle', 'rb')) # прочитать пикл

In [12]:
from objectnat import get_adj_matrix_gdf_to_gdf, get_intermodal_graph
import geopandas as gpd
import os
import pandas as pd
from shapely.ops import unary_union

# Importing necessary libraries:
# - 'objectnat' provides the graph and adjacency matrix calculation tools
# - 'geopandas' for handling geospatial data
# - 'pandas' for working with tabular data
# - 'shapely.ops.unary_union' to combine geometries

buildings = gpd.read_file(os.path.join(data_path, 'buildings.geojson'))
services = gpd.read_file(os.path.join(data_path, 'skateparks.geojson'))
# Reading the building and service data from .parquet files into GeoDataFrames

polygon = unary_union([buildings.geometry.to_list() + services.geometry.to_list()]).convex_hull.buffer(
    0.001)
# Creating a union of all building and service geometries.
# Then generating a convex hull around these geometries and buffering it slightly (0.001 units) 
# to create an encircling polygon that encompasses the data for later use.

#G_intermodal = get_intermodal_graph(polygon=polygon, clip_by_bounds=True)
# Using the encircling polygon to download and clip an intermodal transportation network graph
# 'get_intermodal_graph' fetches a multimodal graph (e.g., walking, public transport) within the given polygon bounds
# you can also use your custom polygon or osm_id of the territory

buildings.to_crs(G_intermodal.graph['crs'], inplace=True)
services.to_crs(G_intermodal.graph['crs'], inplace=True)
# Reprojecting the 'buildings' and 'services' GeoDataFrames to match the coordinate reference system (CRS) of the graph

In [13]:
matrix: pd.DataFrame = get_adj_matrix_gdf_to_gdf(gdf_from=buildings,
                                                 gdf_to=services,
                                                 nx_graph=G_intermodal,
                                                 weight="time_min"
                                                 )
# Creating an adjacency matrix that shows the shortest travel time in minutes ('time_min')
# between each building and service, based on the intermodal graph.
# 'get_adj_matrix_gdf_to_gdf' takes the buildings (gdf_from), services (gdf_to),
# and the graph (nx_graph) and calculates travel times as edge weights.

In [28]:
max_value = matrix.to_numpy().max()
min_value = matrix.to_numpy().min()
print("Максимальное значение в DataFrame:", max_value)
print("Минимальное значение в DataFrame:", min_value)

Максимальное значение в DataFrame: 362.2
Минимальное значение в DataFrame: 0.0


In [29]:
first_matrix: pd.DataFrame = get_adj_matrix_gdf_to_gdf(gdf_from=buildings,
                                                 gdf_to=services,
                                                 nx_graph=G_intermodal,
                                                 weight="time_min",
                                                 threshold=30
                                                 )

In [30]:
max_value = first_matrix.to_numpy().max()
min_value = first_matrix.to_numpy().min()
print("Максимальное значение в DataFrame:", max_value)
print("Минимальное значение в DataFrame:", min_value)

Максимальное значение в DataFrame: inf
Минимальное значение в DataFrame: 0.0


In [31]:
first_matrix

,0,1,2,3,4,5,6,7,8,9,...,68,69,70,71,72,73,74,75,76,77
0,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,28.890625
1,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,29.68750,inf,inf,inf,inf,inf,28.156250
2,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,29.81250,inf,inf,inf,inf,inf,28.140625
3,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,29.59375,inf,inf,inf,inf,inf,28.062500
4,inf,inf,inf,inf,inf,inf,inf,29.84375,inf,inf,...,inf,inf,inf,28.68750,inf,inf,inf,inf,inf,27.015625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36496,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,17.906250,inf,inf,inf
36497,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,17.734375,inf,inf,inf
36498,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,17.406250,inf,inf,inf
36499,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,11.093750,inf,inf,inf


In [ ]:
matrix.to_parquet('F:/Резерв/Учеба/НИР-3/Чат gpt/matrix_time.parquet')
# Saving the resulting adjacency matrix to a .parquet file for future use.

In [24]:
matrix.to_csv('F:/Резерв/Учеба/НИР-3/Чат gpt/matrix_time.csv')
# Saving the resulting adjacency matrix to a .csv file for future use.

In [25]:
loaded_df = pd.read_csv('F:/Резерв/Учеба/НИР-3/Чат gpt/matrix_time.csv', index_col=0)